# TinyStories Story Infilling Model Demo with BPE Tokenizer

This notebook demonstrates how to use the TinyStories story infilling model with the BPE tokenizer. The model takes the first and last sentences of a story as input and generates the middle part.

In [1]:
import torch
import os
import sys
from datasets import load_dataset
import random

# Add project directory to path to import modules
current_path = os.path.abspath('.')
project_name = 'TinyStoriesProject'
project_path = os.path.join(current_path.split(project_name)[0], project_name)
sys.path.append(project_path)
print(f"Project path: {project_path}")

/Users/shawn/Library/Caches/pypoetry/virtualenvs/tinystoriesproject-PsthS1z7-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Project path: /Users/shawn/Documents/sjsu/2025-1/DL_CMPE258/TinyStoriesProject


In [2]:
# Import project modules
from src.models import StoryInfillingModel
from src.bpe_tokenizer import BPETokenizerWrapper
from src.generate_story import generate_story

## 1. Load the pre-trained model

First, let's load the pre-trained model from the saved checkpoint.

In [ ]:
# Check if model exists
model_path = os.path.join(project_path, 'models', 'tinystories_bpe_infilling_model_emb256_layer4_head4_seq256.pth')
if not os.path.exists(model_path):
    print("Model file not found. You need to train the model first by running src/train_infilling_model.py")
else:
    print(f"Model found at {model_path}")

Model found at /Users/shawn/Documents/sjsu/2025-1/DL_CMPE258/TinyStoriesProject/models/tinystories_bpe_infilling_model.pth


In [32]:
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [33]:
# Load model checkpoint
checkpoint = torch.load(model_path, map_location=device)
model_args = checkpoint['args']
tokenizer_model_name = checkpoint.get('tokenizer_model', 'gpt2')

print(f"Model was trained for {checkpoint['epoch']} epochs")
print(f"Train loss: {checkpoint['train_loss']:.4f}, Validation loss: {checkpoint['valid_loss']:.4f}")
print(f"Using tokenizer model: {tokenizer_model_name}")

Model was trained for 7 epochs
Train loss: 1.1038, Validation loss: 1.0846
Using tokenizer model: gpt2


In [34]:
# Initialize BPE tokenizer
tokenizer = BPETokenizerWrapper(
    model_name=tokenizer_model_name,
    special_tokens={"blank_token": "<blank>"}
)
vocab_size = tokenizer.get_vocab_size()
print(f"Tokenizer vocabulary size: {vocab_size}")

Successfully loaded tokenizer
Tokenizer vocabulary size: 50258


In [35]:
# Initialize model
model = StoryInfillingModel(
    vocab_size=vocab_size,
    embed_dim=model_args['embed_dim'],
    num_layers=model_args['num_layers'],
    num_heads=model_args['num_heads'],
    ff_dim=model_args['ff_dim'],
    max_seq_length=model_args['max_seq_length'],
    dropout=model_args['dropout'],
    pad_token_id=tokenizer.pad_token_id,
    blank_token_id=tokenizer.blank_token_id
).to(device)

# Load model weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


## 2. Testing with examples from the validation set

Let's load the validation set and test our model with some real examples.

In [36]:
# Load validation dataset
valid_dataset = load_dataset("roneneldan/TinyStories", split="validation")
print(f"Validation dataset loaded with {len(valid_dataset)} examples")

Validation dataset loaded with 21990 examples


In [37]:
def extract_first_last_sentences(text):
    """Extract the first and last sentences from a story."""
    # Simple sentence splitting by period
    sentences = text.split('.')
    sentences = [s.strip() + '.' for s in sentences if s.strip()]
    
    if len(sentences) < 2:
        return None, None
    
    return sentences[0], sentences[-1]

In [38]:
# Test with a random example from the validation set
def test_with_random_example():
    # Get a random example
    idx = random.randint(0, len(valid_dataset) - 1)
    story = valid_dataset[idx]['text']
    
    # Extract first and last sentences
    first_sentence, last_sentence = extract_first_last_sentences(story)
    
    if not first_sentence or not last_sentence:
        print("Couldn't extract sentences properly. Trying another example.")
        return test_with_random_example()
    
    print("Original story:")
    print("-" * 80)
    print(story)
    print("-" * 80)
    print()
    
    print("First sentence:")
    print(first_sentence)
    print()
    
    print("Last sentence:")
    print(last_sentence)
    print()
    
    # Generate the middle part with our model
    generated_story = model.generate(
        first_sentence,
        last_sentence,
        tokenizer,
        max_length=150,
        teacher_forcing_ratio=0.0  # During testing, we don't use teacher forcing
    )
    
    print("Generated story:")
    print("-" * 80)
    print(generated_story)
    print("-" * 80)
    
    return first_sentence, last_sentence, story, generated_story

In [39]:
first_sentence, last_sentence, original_story, generated_story = test_with_random_example()

Original story:
--------------------------------------------------------------------------------
Once upon a time there was a small class. The class had lots of fun things to do. Every day a teacher came to teach them.

The teacher was very kind, and helped the small class learn about lots of different things. They learned about numbers and animals and colours. 

The teacher also taught them how to read, write and draw. The class was so excited! Every day they loved learning something new.

At the end of each day the teacher gave each student a big hug and said goodbye. 

The small class and the teacher had lots of fun teaching and learning together!
--------------------------------------------------------------------------------

First sentence:
Once upon a time there was a small class.

Last sentence:
The small class and the teacher had lots of fun teaching and learning together!.

Generated story:
--------------------------------------------------------------------------------
Once 

## 3. Custom examples

Now let's try with our own custom first and last sentences.

In [15]:
def generate_with_custom_input(first_sentence, last_sentence, max_tokens=150):
    print("First sentence:")
    print(first_sentence)
    print()
    
    print("Last sentence:")
    print(last_sentence)
    print()
    
    # Generate the middle part with our model
    generated_story = model.generate(
        first_sentence,
        last_sentence,
        tokenizer,
        max_length=max_tokens,
        teacher_forcing_ratio=0.0
    )
    
    print("Generated story:")
    print("-" * 80)
    print(generated_story)
    print("-" * 80)
    
    return generated_story

In [16]:
# Example 1
custom_first_1 = "Once upon a time, there was a little boy named Tim who loved to play with toys."
custom_last_1 = "Tim learned that sharing his toys made everyone happy, including himself."

generated_story_1 = generate_with_custom_input(custom_first_1, custom_last_1)

First sentence:
Once upon a time, there was a little boy named Tim who loved to play with toys.

Last sentence:
Tim learned that sharing his toys made everyone happy, including himself.

Generated story:
--------------------------------------------------------------------------------
Once upon a time, there was a little boy named Tim who loved to play with toys.  Tim From It They From Tim Tim Tim They " They They And " " And They They " And Tim The Tim Tim They They Tim They The They They Tim Tim They " They ". They They They Tim They They They They They They It They Tim They They He ". The They It They They Tim They They They And They They They They They The They They They He They And And And They The They They They They They The The They They The He The The Mom Tim learned that sharing his toys made everyone happy, including himself.
--------------------------------------------------------------------------------


In [17]:
# Example 2
custom_first_2 = "Sarah was excited to visit the zoo with her family on Saturday."
custom_last_2 = "They all agreed it was the best day ever and couldn't wait to come back."

generated_story_2 = generate_with_custom_input(custom_first_2, custom_last_2)

First sentence:
Sarah was excited to visit the zoo with her family on Saturday.

Last sentence:
They all agreed it was the best day ever and couldn't wait to come back.

Generated story:
--------------------------------------------------------------------------------
Sarah was excited to visit the zoo with her family on Saturday.  He She It She She Now She " And ". Even She Now She The She She She She She She From She It She " From She Now Even Now She ". She She She She She She It She Now She She She " She She She That She Her You You What She They And They Together That She They They " They They It They She The She They They You Mom Grand But And They From They Mom You They It You And They She They They Grand The They They They all agreed it was the best day ever and couldn't wait to come back.
--------------------------------------------------------------------------------


In [18]:
# Example 3
custom_first_3 = "It was a rainy day and Max was feeling sad because he couldn't go outside to play."
custom_last_3 = "Max realized that rainy days could be fun too."

generated_story_3 = generate_with_custom_input(custom_first_3, custom_last_3)

First sentence:
It was a rainy day and Max was feeling sad because he couldn't go outside to play.

Last sentence:
Max realized that rainy days could be fun too.

Generated story:
--------------------------------------------------------------------------------
It was a rainy day and Max was feeling sad because he couldn't go outside to play.  He Max It Max He Then It His He He In The He He He He He Max The Now It He He They As As He In From From They He What It He Max " In So That They They It They From They They From They It They The It Max He He They Now From And Mom They And He Grand They They They The They So He They And They They He He " They He He The They They But It But They They " The The They They They They Max realized that rainy days could be fun too.
--------------------------------------------------------------------------------


## 4. Experiment with generation parameters

Let's try changing the generation parameters to see how they affect the output.

In [19]:
def generate_with_parameters(first_sentence, last_sentence, max_tokens=150, temperature=1.0, top_k=50, top_p=0.9):
    # Initialize model for story generation with different parameters
    # Temperature controls randomness: higher values (>1.0) make output more random, lower values (<1.0) make it more deterministic
    
    print(f"Generating with temperature={temperature}, top_k={top_k}, top_p={top_p}")
    
    # For direct use of the model's generate method with more control
    generated_story = model.generate(
        first_sentence,
        last_sentence,
        tokenizer,
        max_length=max_tokens,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        teacher_forcing_ratio=0.0
    )
    
    print("Generated story:")
    print("-" * 80)
    print(generated_story)
    print("-" * 80)
    
    return generated_story

In [20]:
# Choose an example
test_first = "Jake was a little boy who always wanted a puppy for his birthday."
test_last = "Jake was so happy with his new puppy and promised to take care of it forever."

print("First sentence:")
print(test_first)
print()
print("Last sentence:")
print(test_last)
print()

First sentence:
Jake was a little boy who always wanted a puppy for his birthday.

Last sentence:
Jake was so happy with his new puppy and promised to take care of it forever.



In [21]:
# Generate with different temperatures
low_temp_story = generate_with_parameters(test_first, test_last, temperature=0.5)
normal_temp_story = generate_with_parameters(test_first, test_last, temperature=1.0)
high_temp_story = generate_with_parameters(test_first, test_last, temperature=1.5)

Generating with temperature=0.5, top_k=50, top_p=0.9
Generated story:
--------------------------------------------------------------------------------
Jake was a little boy who always wanted a puppy for his birthday.  He He He He He He He He He He He He He He He He He He He He He He He He Now He He He He He He From He He He From He He He He He He He He He He He He He You He He He He He Hed Hed He From He He He He He He He He He He He He He He He Hed He Hed He He He He Hedd He He He He Hed He Jake was so happy with his new puppy and promised to take care of it forever.
--------------------------------------------------------------------------------
Generating with temperature=1.0, top_k=50, top_p=0.9
Generated story:
--------------------------------------------------------------------------------
Jake was a little boy who always wanted a puppy for his birthday.  He But He He He They From He The It He Now And He He ". The He He You His He He He He From Jake Everyone He From He Now In Now

In [22]:
# Generate with different top_k and top_p values
low_k_story = generate_with_parameters(test_first, test_last, top_k=10)
high_k_story = generate_with_parameters(test_first, test_last, top_k=100)
low_p_story = generate_with_parameters(test_first, test_last, top_p=0.5)
high_p_story = generate_with_parameters(test_first, test_last, top_p=0.99)

Generating with temperature=1.0, top_k=10, top_p=0.9
Generated story:
--------------------------------------------------------------------------------
Jake was a little boy who always wanted a puppy for his birthday.  " He He He It He Now The They Jake They The He He He The From He The He Now " " Now He He The The He He He You The ". He You He Jake He Now He He He You The Now He " He Now From You He You You He They The He The He He From From He They You The It The He He The You He It It From They He The They He They They It The They They They They They They They They Jake was so happy with his new puppy and promised to take care of it forever.
--------------------------------------------------------------------------------
Generating with temperature=1.0, top_k=100, top_p=0.9
Generated story:
--------------------------------------------------------------------------------
Jake was a little boy who always wanted a puppy for his birthday.  He He With She He The The They The They From He 

## 5. Trying teacher forcing during generation

Let's see how teacher forcing affects generation when we have a ground truth story.

In [23]:
def generate_with_teacher_forcing(first_sentence, last_sentence, ground_truth, ratio=0.5):
    print(f"Generating with teacher_forcing_ratio={ratio}")
    print("Ground truth:")
    print("-" * 80)
    print(ground_truth)
    print("-" * 80)
    print()
    
    # Generate with teacher forcing
    generated_story = model.generate(
        first_sentence,
        last_sentence,
        tokenizer,
        max_length=200,
        teacher_forcing_ratio=ratio,
        ground_truth=ground_truth
    )
    
    print("Generated story:")
    print("-" * 80)
    print(generated_story)
    print("-" * 80)
    
    return generated_story

In [24]:
# Get a random story from the validation set
idx = random.randint(0, len(valid_dataset) - 1)
ground_truth = valid_dataset[idx]['text']
first_sentence, last_sentence = extract_first_last_sentences(ground_truth)

if not first_sentence or not last_sentence:
    print("Couldn't extract sentences properly. Please run this cell again.")
else:
    # Generate with different teacher forcing ratios
    no_tf_story = generate_with_teacher_forcing(first_sentence, last_sentence, ground_truth, ratio=0.0)
    half_tf_story = generate_with_teacher_forcing(first_sentence, last_sentence, ground_truth, ratio=0.5)
    full_tf_story = generate_with_teacher_forcing(first_sentence, last_sentence, ground_truth, ratio=1.0)

Generating with teacher_forcing_ratio=0.0
Ground truth:
--------------------------------------------------------------------------------
Once upon a time there was an ugly earth. It was so sad and alone. One day, a magical creature appeared and said, "I can help you feel better."

The earth said, "What can you do?"

The creature said, "I can give you something to drink."

The earth said, "What can I drink?"

The creature said, "A magical drink that will make you feel better."

So the earth drank the magical drink and sure enough it felt better. The earth was so happy that it said, "Thank you for the magical drink. You made me feel better."

The creature smiled and said, "You're very welcome." And then it flew away.

From that day on, the earth was happy and not so ugly anymore.
--------------------------------------------------------------------------------

Generated story:
--------------------------------------------------------------------------------
Once upon a time there was an u

## 6. Continuation Generation (Without Last Sentence)

The model can also generate continuations when only given the first sentence.

In [25]:
def generate_continuation(first_sentence, max_tokens=200):
    print("First sentence:")
    print(first_sentence)
    print()
    
    # Generate continuation with our model
    generated_story = model.generate(
        first_sentence,
        last_sentence=None,  # No last sentence for continuation
        tokenizer=tokenizer,
        max_length=max_tokens,
        teacher_forcing_ratio=0.0
    )
    
    print("Generated continuation:")
    print("-" * 80)
    print(generated_story)
    print("-" * 80)
    
    return generated_story

In [26]:
# Example continuations
custom_first_1 = "Once upon a time, there was a little girl named Lily who loved to dance in the rain."
continuation_1 = generate_continuation(custom_first_1)

custom_first_2 = "Tim got a new robot toy for his birthday and was very excited to show it to his friends."
continuation_2 = generate_continuation(custom_first_2)

First sentence:
Once upon a time, there was a little girl named Lily who loved to dance in the rain.

Generated continuation:
--------------------------------------------------------------------------------
Once upon a time, there was a little girl named Lily who loved to dance in the rain. <blank> The moral of the story is that it's always better to have fun and be kind to others.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
--------------------------------------------------------------------------------
First sentence:
Tim got a new robot toy for his birthday and was very excited to show it to his friends.

Generated continuation:
--------------------------------------------------------------------------------
Tim got a new robot toy for his birthday and was very excited to show it to his friends. <blank> He was never able to ride his bike again.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
--